# 🚀 YOLOv8 with CBAM Attention - Google Colab Training
## Bangladesh Road Vehicle Detection

**Model**: YOLOv8n with CBAM (Convolutional Block Attention Module)

**Dataset**: 10 classes - bicycle, bus, car, CNG, auto, bike, Multi-Class, rickshaw, truck, van

**Hardware**: Google Colab T4 GPU (Free)

**Expected Training Time**: 3-4 hours for 100 epochs

---

### 📋 Instructions:
1. Run cells in order from top to bottom
2. Don't skip any cells
3. Wait for each cell to complete before running the next
4. Check for ✓ marks indicating success

## 📁 Step 1: Mount Google Drive

This will ask for permission to access your Google Drive.

In [1]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

print('\n' + '='*70)
print('✓ Google Drive mounted successfully!')
print('='*70)

Mounted at /content/drive

✓ Google Drive mounted successfully!


## ⚙️ Step 2: Install Dependencies

Install Ultralytics YOLOv8 and required packages.

In [2]:
%%capture
# Install ultralytics (output suppressed for cleaner display)
!pip install ultralytics==8.0.200 --quiet

print('Installing dependencies...')

In [3]:
# Verify installation
import sys
import torch
import ultralytics
from ultralytics import YOLO

print('='*70)
print('ENVIRONMENT VERIFICATION')
print('='*70)
print(f'✓ Python version: {sys.version_info.major}.{sys.version_info.minor}.{sys.version_info.micro}')
print(f'✓ PyTorch version: {torch.__version__}')
print(f'✓ Ultralytics version: {ultralytics.__version__}')
print(f'✓ CUDA available: {torch.cuda.is_available()}')

if torch.cuda.is_available():
    print(f'✓ CUDA version: {torch.version.cuda}')
    print(f'✓ GPU: {torch.cuda.get_device_name(0)}')
    print(f'✓ GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB')
else:
    print('⚠ WARNING: CUDA not available. Training will be very slow on CPU!')

print('='*70)

ENVIRONMENT VERIFICATION
✓ Python version: 3.12.12
✓ PyTorch version: 2.8.0+cu126
✓ Ultralytics version: 8.0.200
✓ CUDA available: True
✓ CUDA version: 12.6
✓ GPU: Tesla T4
✓ GPU Memory: 14.7 GB


## 📂 Step 3: Configure Paths

**⚠️ IMPORTANT**: Update `DRIVE_PROJECT_PATH` to match your Google Drive folder structure.

In [4]:
import os
from pathlib import Path

# ============================================================================
# ⚠️ UPDATE THIS PATH TO YOUR GOOGLE DRIVE FOLDER
# ============================================================================
DRIVE_PROJECT_PATH = '/content/drive/MyDrive/YOLOv8 Traffic'

# Dataset path (from data.yaml)
DATASET_PATH = f'{DRIVE_PROJECT_PATH}/dataset'

# File paths
DATA_YAML = f'{DRIVE_PROJECT_PATH}/data.yaml'
PRETRAINED_WEIGHTS = 'yolov8n.pt'  # Will be downloaded automatically

# Output paths (on Colab, will be copied to Drive later)
COLAB_WORK_DIR = '/content/yolov8_cbam'
RESULTS_DIR = '/content/runs'

# Create working directory
os.makedirs(COLAB_WORK_DIR, exist_ok=True)

# Verify paths
print('='*70)
print('PATH CONFIGURATION')
print('='*70)
print(f'Drive project: {DRIVE_PROJECT_PATH}')
print(f'Dataset path: {DATASET_PATH}')
print(f'Data config: {DATA_YAML}')
print(f'Working dir: {COLAB_WORK_DIR}')
print(f'Results dir: {RESULTS_DIR}')
print('='*70)

# Verify critical files exist
print('\nVerifying files...')
if os.path.exists(DATA_YAML):
    print(f'✓ data.yaml found')
else:
    print(f'✗ ERROR: data.yaml not found at {DATA_YAML}')
    print('  Please check your DRIVE_PROJECT_PATH setting above!')
    raise FileNotFoundError(f'data.yaml not found at {DATA_YAML}')

PATH CONFIGURATION
Drive project: /content/drive/MyDrive/YOLOv8 Traffic
Dataset path: /content/drive/MyDrive/YOLOv8 Traffic/dataset
Data config: /content/drive/MyDrive/YOLOv8 Traffic/data.yaml
Working dir: /content/yolov8_cbam
Results dir: /content/runs

Verifying files...
✓ data.yaml found


## 🧬 Step 4: Define CBAM Module

This cell defines the CBAM (Convolutional Block Attention Module) with comprehensive error handling.

In [5]:
"""
CBAM: Convolutional Block Attention Module
Reference: Woo et al., ECCV 2018
Implementation optimized for Google Colab with T4 GPU
"""

import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import Optional
import warnings


class ChannelAttention(nn.Module):
    """Channel Attention Module - focuses on 'what' is meaningful."""

    def __init__(self, in_channels: int, reduction_ratio: int = 16):
        super(ChannelAttention, self).__init__()

        if in_channels <= 0:
            raise ValueError(f"in_channels must be positive, got {in_channels}")
        if reduction_ratio <= 0:
            raise ValueError(f"reduction_ratio must be positive, got {reduction_ratio}")

        self.reduced_channels = max(in_channels // reduction_ratio, 1)

        # Shared MLP using 1x1 convolutions (more efficient than FC)
        self.fc1 = nn.Conv2d(in_channels, self.reduced_channels, 1, bias=False)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Conv2d(self.reduced_channels, in_channels, 1, bias=False)

        # Pooling layers
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)

        # Initialize weights
        self._initialize_weights()

    def _initialize_weights(self):
        """Kaiming initialization for ReLU activation."""
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        if not isinstance(x, torch.Tensor):
            raise TypeError(f"Expected torch.Tensor, got {type(x)}")
        if x.dim() != 4:
            raise ValueError(f"Expected 4D input (B,C,H,W), got shape {x.shape}")

        # Average pooling branch
        avg_out = self.fc2(self.relu(self.fc1(self.avg_pool(x))))

        # Max pooling branch
        max_out = self.fc2(self.relu(self.fc1(self.max_pool(x))))

        # Combine and apply sigmoid
        out = torch.sigmoid(avg_out + max_out)

        return out


class SpatialAttention(nn.Module):
    """Spatial Attention Module - focuses on 'where' is informative."""

    def __init__(self, kernel_size: int = 7):
        super(SpatialAttention, self).__init__()

        if kernel_size <= 0 or kernel_size % 2 == 0:
            raise ValueError(f"kernel_size must be positive and odd, got {kernel_size}")

        padding = kernel_size // 2

        # 7x7 convolution to aggregate channel information
        self.conv = nn.Conv2d(2, 1, kernel_size, padding=padding, bias=False)

        # Initialize weights
        self._initialize_weights()

    def _initialize_weights(self):
        """Xavier initialization for sigmoid activation."""
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_normal_(m.weight)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        if not isinstance(x, torch.Tensor):
            raise TypeError(f"Expected torch.Tensor, got {type(x)}")
        if x.dim() != 4:
            raise ValueError(f"Expected 4D input (B,C,H,W), got shape {x.shape}")

        # Aggregate channel information
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)

        # Concatenate and apply convolution + sigmoid
        concat = torch.cat([avg_out, max_out], dim=1)
        out = torch.sigmoid(self.conv(concat))

        return out


class CBAM(nn.Module):
    """
    Complete CBAM module combining Channel and Spatial Attention.

    Args:
        c1: Input channels (will be auto-scaled by YOLOv8)
        c2: Output channels (ignored, CBAM preserves channels)
        reduction_ratio: Channel reduction ratio for efficient attention
        kernel_size: Kernel size for spatial attention convolution
        shortcut: Whether to use residual connection

    Note: YOLOv8 automatically scales c1 based on model variant (n/s/m/l/x).
          For 'n' variant with scale [0.33, 0.25, 1024], c1 will be scaled by 0.25.
    """

    def __init__(
        self,
        c1: int,
        c2: Optional[int] = None,
        reduction_ratio: int = 16,
        kernel_size: int = 7,
        shortcut: bool = True
    ):
        super(CBAM, self).__init__()

        # Validate inputs
        if not isinstance(c1, int) or c1 <= 0:
            raise ValueError(f"c1 must be a positive integer, got {c1} (type: {type(c1)})")
        if reduction_ratio <= 0:
            raise ValueError(f"reduction_ratio must be positive, got {reduction_ratio}")
        if kernel_size <= 0 or kernel_size % 2 == 0:
            raise ValueError(f"kernel_size must be positive and odd, got {kernel_size}")

        # CBAM doesn't change channel dimensions - it's an attention mechanism
        # c2 is provided by YOLOv8 parser but should equal c1
        if c2 is not None and c2 != c1:
            warnings.warn(
                f"CBAM is attention module - doesn't change channels. "
                f"c2={c2} ignored, output will have c1={c1} channels."
            )

        # Store configuration
        self.c1 = c1  # Input channels (already scaled by YOLOv8)
        self.c2 = c1  # Output channels (same as input for attention)
        self.shortcut = shortcut
        self.reduction_ratio = reduction_ratio

        # Build attention modules with ACTUAL input channels
        self.channel_attention = ChannelAttention(c1, reduction_ratio)
        self.spatial_attention = SpatialAttention(kernel_size)

        # Debug info (will be printed during model construction)
        print(f"  CBAM initialized: c1={c1}, reduction_ratio={reduction_ratio}, shortcut={shortcut}")

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Apply CBAM attention to input tensor.

        Args:
            x: Input tensor of shape (B, C, H, W)

        Returns:
            Attended tensor of shape (B, C, H, W)
        """
        # Comprehensive input validation
        if not isinstance(x, torch.Tensor):
            raise TypeError(f"Expected torch.Tensor input, got {type(x)}")

        if x.dim() != 4:
            raise ValueError(f"Expected 4D input (B,C,H,W), got {x.dim()}D with shape {x.shape}")

        # Verify channel count matches initialization
        actual_channels = x.size(1)
        if actual_channels != self.c1:
            raise ValueError(
                f"Channel mismatch! CBAM was initialized with c1={self.c1} channels, "
                f"but received input with {actual_channels} channels. "
                f"Input shape: {x.shape}. "
                f"This usually means the YAML architecture has incorrect channel specifications."
            )

        # Check for invalid values
        if torch.isnan(x).any():
            raise RuntimeError(f"Input contains NaN values! Shape: {x.shape}")
        if torch.isinf(x).any():
            raise RuntimeError(f"Input contains Inf values! Shape: {x.shape}")

        # Store identity for residual connection
        identity = x

        # Apply channel attention: emphasize important channels
        x = x * self.channel_attention(x)

        # Apply spatial attention: emphasize important spatial regions
        x = x * self.spatial_attention(x)

        # Add residual connection (helps gradient flow and preserves information)
        if self.shortcut:
            x = x + identity

        # Final validation
        if torch.isnan(x).any():
            raise RuntimeError("CBAM output contains NaN values!")
        if torch.isinf(x).any():
            raise RuntimeError("CBAM output contains Inf values!")

        return x


# Quick test
print('='*70)
print('CBAM MODULE DEFINED')
print('='*70)
print('Testing CBAM module...')

try:
    test_cbam = CBAM(c1=64)
    test_input = torch.randn(1, 64, 32, 32)

    if torch.cuda.is_available():
        test_cbam = test_cbam.cuda()
        test_input = test_input.cuda()

    with torch.no_grad():
        test_output = test_cbam(test_input)

    assert test_output.shape == test_input.shape
    assert not torch.isnan(test_output).any()
    assert not torch.isinf(test_output).any()

    print(f'✓ CBAM test passed!')
    print(f'  Input shape: {tuple(test_input.shape)}')
    print(f'  Output shape: {tuple(test_output.shape)}')
    print(f'  Parameters: {sum(p.numel() for p in test_cbam.parameters()):,}')
    print(f'  Device: {"GPU" if torch.cuda.is_available() else "CPU"}')

except Exception as e:
    print(f'✗ CBAM test failed: {e}')
    raise

print('='*70)

CBAM MODULE DEFINED
Testing CBAM module...
  CBAM initialized: c1=64, reduction_ratio=16, shortcut=True
✓ CBAM test passed!
  Input shape: (1, 64, 32, 32)
  Output shape: (1, 64, 32, 32)
  Parameters: 610
  Device: GPU


## 🔗 Step 5: Register CBAM with Ultralytics

Register the CBAM module so YOLOv8 can use it.

In [6]:
from ultralytics.nn import tasks

# Register CBAM module
tasks.CBAM = CBAM

print('='*70)
print('CBAM REGISTRATION')
print('='*70)

# Verify registration
if hasattr(tasks, 'CBAM'):
    print('✓ CBAM successfully registered with Ultralytics')
    print('✓ YOLOv8 can now use CBAM modules in model architecture')
else:
    print('✗ CBAM registration failed!')
    raise RuntimeError('CBAM module registration failed')

print('='*70)

CBAM REGISTRATION
✓ CBAM successfully registered with Ultralytics
✓ YOLOv8 can now use CBAM modules in model architecture


## 🏗️ Step 6: Create Model Architecture

Define YOLOv8n-CBAM architecture in YAML format.

In [10]:
import yaml

# YOLOv8n-CBAM architecture definition
# CRITICAL FIX: YOLOv8 does NOT auto-scale custom module arguments!
# We must manually specify the ACTUAL scaled channel counts for CBAM

# For 'n' variant: width_multiple = 0.25
# So: 128*0.25=32, 256*0.25=64, 512*0.25=128, 1024*0.25=256

model_config = {
    'nc': 10,  # Number of classes (Bangladesh road vehicles)
    'scales': {
        # Format: [depth_multiple, width_multiple, max_channels]
        # 'n' variant uses 0.33 depth scaling and 0.25 width scaling
        'n': [0.33, 0.25, 1024]
    },
    'backbone': [
        # [from, repeats, module, args]
        # Standard modules (Conv, C2f) get auto-scaled by YOLOv8
        # Custom modules (CBAM) need ACTUAL scaled values!

        [-1, 1, 'Conv', [64, 3, 2]],   # 0-P1/2
        [-1, 1, 'Conv', [128, 3, 2]],  # 1-P2/4
        [-1, 3, 'C2f', [128, True]],   # 2 → outputs 32 channels (128*0.25)
        [-1, 1, 'CBAM', [32]],         # 3-CBAM → 32 channels (ACTUAL scaled value!)

        [-1, 1, 'Conv', [256, 3, 2]],  # 4-P3/8
        [-1, 6, 'C2f', [256, True]],   # 5 → outputs 64 channels (256*0.25)
        [-1, 1, 'CBAM', [64]],         # 6-CBAM → 64 channels (ACTUAL scaled value!)

        [-1, 1, 'Conv', [512, 3, 2]],  # 7-P4/16
        [-1, 6, 'C2f', [512, True]],   # 8 → outputs 128 channels (512*0.25)
        [-1, 1, 'CBAM', [128]],        # 9-CBAM → 128 channels (ACTUAL scaled value!)

        [-1, 1, 'Conv', [1024, 3, 2]], # 10-P5/32
        [-1, 3, 'C2f', [1024, True]],  # 11 → outputs 256 channels (1024*0.25=256)
        [-1, 1, 'CBAM', [256]],        # 12-CBAM → 256 channels (ACTUAL scaled value!)

        [-1, 1, 'SPPF', [1024, 5]],    # 13 (SPPF layer)
    ],
    'head': [
        # Detection head with feature pyramid
        [-1, 1, 'nn.Upsample', [None, 2, 'nearest']],  # 14
        [[-1, 9], 1, 'Concat', [1]],                   # 15 (concat with CBAM layer 9)
        [-1, 3, 'C2f', [512]],                         # 16 (P4/16-medium)

        [-1, 1, 'nn.Upsample', [None, 2, 'nearest']],  # 17
        [[-1, 6], 1, 'Concat', [1]],                   # 18 (concat with CBAM layer 6)
        [-1, 3, 'C2f', [256]],                         # 19 (P3/8-small)

        [-1, 1, 'Conv', [256, 3, 2]],                  # 20
        [[-1, 16], 1, 'Concat', [1]],                  # 21
        [-1, 3, 'C2f', [512]],                         # 22 (P4/16-medium)

        [-1, 1, 'Conv', [512, 3, 2]],                  # 23
        [[-1, 13], 1, 'Concat', [1]],                  # 24
        [-1, 3, 'C2f', [1024]],                        # 25 (P5/32-large)

        [[19, 22, 25], 1, 'Detect', ['nc']],           # 26 Detect(P3, P4, P5)
    ]
}

# Save model configuration
model_yaml_path = f'{COLAB_WORK_DIR}/yolov8n-cbam.yaml'
with open(model_yaml_path, 'w') as f:
    yaml.dump(model_config, f, default_flow_style=False, sort_keys=False)

print('='*70)
print('MODEL ARCHITECTURE')
print('='*70)
print(f'✓ YOLOv8n-CBAM architecture created')
print(f'✓ Configuration saved to: {model_yaml_path}')
print(f'✓ CBAM modules: 4 (with ACTUAL scaled channels: 32, 64, 128, 256)')
print(f'✓ Number of classes: 10')
print('='*70)
print('\nCBAM channel specifications:')
print('  Layer 3 CBAM: 32 channels  (matches C2f[128] output after 0.25 scaling)')
print('  Layer 6 CBAM: 64 channels  (matches C2f[256] output after 0.25 scaling)')
print('  Layer 9 CBAM: 128 channels (matches C2f[512] output after 0.25 scaling)')
print('  Layer 12 CBAM: 256 channels (matches C2f[1024] output after 0.25 scaling)')
print('='*70)

MODEL ARCHITECTURE
✓ YOLOv8n-CBAM architecture created
✓ Configuration saved to: /content/yolov8_cbam/yolov8n-cbam.yaml
✓ CBAM modules: 4 (with ACTUAL scaled channels: 32, 64, 128, 256)
✓ Number of classes: 10

CBAM channel specifications:
  Layer 3 CBAM: 32 channels  (matches C2f[128] output after 0.25 scaling)
  Layer 6 CBAM: 64 channels  (matches C2f[256] output after 0.25 scaling)
  Layer 9 CBAM: 128 channels (matches C2f[512] output after 0.25 scaling)
  Layer 12 CBAM: 256 channels (matches C2f[1024] output after 0.25 scaling)


## 🔍 Step 7: Validate Dataset Configuration

Check if dataset and configuration are correct before training.

In [11]:
import yaml
from pathlib import Path

print('='*70)
print('DATASET VALIDATION')
print('='*70)

# Load data.yaml
try:
    with open(DATA_YAML, 'r') as f:
        data_config = yaml.safe_load(f)
    print(f'✓ Loaded data.yaml from: {DATA_YAML}')
except Exception as e:
    print(f'✗ Failed to load data.yaml: {e}')
    raise

# Validate required fields
required_fields = ['path', 'train', 'val', 'nc', 'names']
missing_fields = []

for field in required_fields:
    if field in data_config:
        print(f'✓ Field "{field}" present')
    else:
        print(f'✗ Missing required field: "{field}"')
        missing_fields.append(field)

if missing_fields:
    raise ValueError(f'Missing required fields in data.yaml: {missing_fields}')

# Validate number of classes
nc = data_config['nc']
names = data_config['names']
num_names = len(names) if isinstance(names, (list, dict)) else 0

if nc == num_names:
    print(f'✓ Number of classes matches: {nc}')
else:
    print(f'✗ Class mismatch: nc={nc} but found {num_names} names')
    raise ValueError(f'Number of classes mismatch')

# Display class names
print(f'\nClasses ({nc}):')
if isinstance(names, dict):
    for idx, name in names.items():
        print(f'  {idx}: {name}')
elif isinstance(names, list):
    for idx, name in enumerate(names):
        print(f'  {idx}: {name}')

# Check dataset path
dataset_path = data_config['path']
print(f'\nDataset path: {dataset_path}')

if os.path.exists(dataset_path):
    print(f'✓ Dataset path exists')

    # Check train/val directories
    train_images = Path(dataset_path) / data_config['train']
    val_images = Path(dataset_path) / data_config['val']

    if train_images.exists():
        train_count = len(list(train_images.glob('*.jpg'))) + len(list(train_images.glob('*.png')))
        print(f'✓ Training images path exists: {train_count} images found')
    else:
        print(f'⚠ Training images path not found: {train_images}')

    if val_images.exists():
        val_count = len(list(val_images.glob('*.jpg'))) + len(list(val_images.glob('*.png')))
        print(f'✓ Validation images path exists: {val_count} images found')
    else:
        print(f'⚠ Validation images path not found: {val_images}')
else:
    print(f'⚠ Dataset path not accessible from Colab')
    print(f'  This may be OK if the path is relative and will be resolved during training')

print('='*70)

DATASET VALIDATION
✓ Loaded data.yaml from: /content/drive/MyDrive/YOLOv8 Traffic/data.yaml
✓ Field "path" present
✓ Field "train" present
✓ Field "val" present
✓ Field "nc" present
✓ Field "names" present
✓ Number of classes matches: 10

Classes (10):
  0: bicycle
  1: bus
  2: car
  3: cng
  4: auto
  5: bike
  6: Multi-Class
  7: rickshaw
  8: truck
  9: van

Dataset path: /content/drive/MyDrive/YOLOv8 Traffic/dataset
✓ Dataset path exists
✓ Training images path exists: 868 images found
✓ Validation images path exists: 217 images found


## 🏋️ Step 8: Load and Validate Model

Load the YOLOv8n-CBAM model and pretrained weights.

In [13]:
import torch.serialization

# Add safe globals for PyTorch 2.6 compatibility
torch.serialization.add_safe_globals(['ultralytics.nn.tasks.DetectionModel'])

print('='*70)
print('MODEL LOADING')
print('='*70)

try:
    # Load model architecture
    print(f'Loading model from: {model_yaml_path}')
    model = YOLO(model_yaml_path)
    print('✓ Model architecture loaded')

    # Load pretrained weights (with PyTorch 2.6 compatibility)
    print(f'\nLoading pretrained weights: {PRETRAINED_WEIGHTS}')
    try:
        model = model.load(PRETRAINED_WEIGHTS)
    except Exception as weight_error:
        # If weights_only causes issues, try with explicit setting
        print('  Retrying with adjusted torch.load settings...')
        import torch
        original_weights_only = getattr(torch.serialization, '_use_new_zipfile_serialization', None)
        try:
            # Temporarily allow non-weights-only loading for trusted ultralytics weights
            model.ckpt = None
            from ultralytics.nn.tasks import attempt_load_one_weight
            # Patch torch.load temporarily
            _original_load = torch.load
            def _patched_load(f, map_location=None, *args, **kwargs):
                kwargs['weights_only'] = False
                return _original_load(f, map_location=map_location, *args, **kwargs)
            torch.load = _patched_load
            model = model.load(PRETRAINED_WEIGHTS)
            torch.load = _original_load
        except Exception as e:
            torch.load = _original_load  # Restore original
            raise e

    print('✓ Pretrained weights loaded (transfer learning enabled)')

    # Display model info
    print('\n' + '='*70)
    print('MODEL INFORMATION')
    print('='*70)
    model.info(verbose=False)

    # Test forward pass
    print('\n' + '='*70)
    print('FORWARD PASS TEST')
    print('='*70)

    device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
    model.model.to(device)
    model.model.eval()

    # Test with 640x640 input
    test_input = torch.randn(1, 3, 640, 640).to(device)

    with torch.no_grad():
        test_output = model.model(test_input)

    print(f'✓ Forward pass successful')
    print(f'  Input shape: {tuple(test_input.shape)}')
    print(f'  Device: {device}')
    print(f'  Memory allocated: {torch.cuda.memory_allocated(0) / 1024**2:.1f} MB' if torch.cuda.is_available() else '  Device: CPU')

    # Set back to training mode
    model.model.train()

    print('\n✓ Model ready for training!')
    print('='*70)

except Exception as e:
    print(f'\n✗ Model loading failed: {e}')
    import traceback
    print('\nFull error traceback:')
    traceback.print_exc()
    raise


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1       226  CBAM                                         [32]                          
  4                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  5                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  6                  -1  1       610  CBAM                                         [64]                          
  7                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64,

MODEL LOADING
Loading model from: /content/yolov8_cbam/yolov8n-cbam.yaml
  CBAM initialized: c1=32, reduction_ratio=16, shortcut=True
  CBAM initialized: c1=64, reduction_ratio=16, shortcut=True
  CBAM initialized: c1=128, reduction_ratio=16, shortcut=True
  CBAM initialized: c1=256, reduction_ratio=16, shortcut=True


YOLOv8n-cbam summary: 261 layers, 3024070 parameters, 3024054 gradients, 8.2 GFLOPs



✓ Model architecture loaded

Loading pretrained weights: yolov8n.pt
  Retrying with adjusted torch.load settings...


Transferred 41/367 items from pretrained weights


✓ Pretrained weights loaded (transfer learning enabled)

MODEL INFORMATION

FORWARD PASS TEST
✓ Forward pass successful
  Input shape: (1, 3, 640, 640)
  Device: cuda:0
  Memory allocated: 19.3 MB

✓ Model ready for training!


## ⚙️ Step 9: Configure Training Parameters

Set up hyperparameters optimized for Google Colab T4 GPU.

In [16]:
# Training configuration optimized for T4 GPU
TRAINING_CONFIG = {
    'data': DATA_YAML,
    'epochs': 100,                    # Number of epochs
    'imgsz': 640,                     # Image size
    'batch': 16,                      # Batch size (optimal for T4 16GB)
    'device': 0,                      # GPU device
    'workers': 2,                     # DataLoader workers (Colab has limited CPU)
    'project': 'yolov8_cbam_training',  # Project name (NOT a path - wandb compatible)
    'name': 'bd_vehicles_run',          # Experiment name
    'exist_ok': True,                 # Overwrite existing project
    'pretrained': True,               # Use pretrained weights
    'optimizer': 'auto',              # SGD for small batches, AdamW for large
    'verbose': True,                  # Verbose output
    'seed': 42,                       # Random seed for reproducibility
    'deterministic': False,           # Faster training (non-deterministic)
    'single_cls': False,              # Multi-class detection
    'rect': False,                    # Rectangular training
    'cos_lr': True,                   # Cosine learning rate scheduler
    'close_mosaic': 10,               # Disable mosaic last N epochs
    'resume': False,                  # Resume from last checkpoint
    'amp': True,                      # Automatic Mixed Precision (faster on T4)
    'fraction': 1.0,                  # Use full dataset
    'profile': False,                 # Don't profile (saves memory)
    'freeze': None,                   # Don't freeze layers
    'save': True,                     # Save checkpoints
    'save_period': -1,                # Save checkpoint every N epochs (-1 = last only)
    'cache': False,                   # Don't cache images (saves RAM)
    'patience': 50,                   # Early stopping patience
    'plots': True,                    # Generate plots
    'val': True,                      # Validate during training

    # Hyperparameters
    'lr0': 0.01,                      # Initial learning rate
    'lrf': 0.01,                      # Final learning rate (lr0 * lrf)
    'momentum': 0.937,                # SGD momentum
    'weight_decay': 0.0005,           # Weight decay
    'warmup_epochs': 3.0,             # Warmup epochs
    'warmup_momentum': 0.8,           # Warmup momentum
    'warmup_bias_lr': 0.1,            # Warmup bias learning rate
    'box': 7.5,                       # Box loss gain
    'cls': 0.5,                       # Class loss gain
    'dfl': 1.5,                       # DFL loss gain
    'label_smoothing': 0.0,           # Label smoothing
    'nbs': 64,                        # Nominal batch size
    'hsv_h': 0.015,                   # HSV-Hue augmentation
    'hsv_s': 0.7,                     # HSV-Saturation augmentation
    'hsv_v': 0.4,                     # HSV-Value augmentation
    'degrees': 0.0,                   # Rotation augmentation
    'translate': 0.1,                 # Translation augmentation
    'scale': 0.5,                     # Scale augmentation
    'shear': 0.0,                     # Shear augmentation
    'perspective': 0.0,               # Perspective augmentation
    'flipud': 0.0,                    # Vertical flip probability
    'fliplr': 0.5,                    # Horizontal flip probability
    'mosaic': 1.0,                    # Mosaic augmentation probability
    'mixup': 0.0,                     # Mixup augmentation probability
    'copy_paste': 0.0,                # Copy-paste augmentation probability
}

# Disable wandb if it's causing issues (optional - uncomment if needed)
import os
os.environ['WANDB_DISABLED'] = 'true'

# Display configuration
print('='*70)
print('TRAINING CONFIGURATION (Optimized for T4 GPU)')
print('='*70)
print(f'Epochs: {TRAINING_CONFIG["epochs"]}')
print(f'Batch size: {TRAINING_CONFIG["batch"]}')
print(f'Image size: {TRAINING_CONFIG["imgsz"]}')
print(f'Device: GPU {TRAINING_CONFIG["device"]} (T4)' if torch.cuda.is_available() else 'CPU')
print(f'Mixed Precision: {TRAINING_CONFIG["amp"]}')
print(f'Learning rate: {TRAINING_CONFIG["lr0"]} → {TRAINING_CONFIG["lr0"] * TRAINING_CONFIG["lrf"]}')
print(f'Early stopping patience: {TRAINING_CONFIG["patience"]} epochs')
print(f'Project: {TRAINING_CONFIG["project"]} / {TRAINING_CONFIG["name"]}')
print(f'Results will be saved to: runs/detect/{TRAINING_CONFIG["project"]}/{TRAINING_CONFIG["name"]}')
print('='*70)

# Memory check for T4
if torch.cuda.is_available():
    total_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f'\nGPU Memory: {total_memory:.1f} GB')

    if total_memory < 15:
        print('⚠ WARNING: Less than 15GB GPU memory detected')
        print('  Consider reducing batch size if you encounter OOM errors')
    else:
        print('✓ GPU memory sufficient for batch size 16')

print('\n✓ Training configuration ready!')
print('✓ Weights & Biases (wandb) disabled for simpler setup')


TRAINING CONFIGURATION (Optimized for T4 GPU)
Epochs: 100
Batch size: 16
Image size: 640
Device: GPU 0 (T4)
Mixed Precision: True
Learning rate: 0.01 → 0.0001
Early stopping patience: 50 epochs
Project: yolov8_cbam_training / bd_vehicles_run
Results will be saved to: runs/detect/yolov8_cbam_training/bd_vehicles_run

GPU Memory: 14.7 GB
⚠ WARNING: Less than 15GB GPU memory detected
  Consider reducing batch size if you encounter OOM errors

✓ Training configuration ready!
✓ Weights & Biases (wandb) disabled for simpler setup


## 🚀 Step 10: Start Training

**This will take approximately 3-4 hours on a T4 GPU for 100 epochs.**

⚠️ **Important**: Make sure your Colab session stays active. Consider:
- Using Colab Pro for longer sessions
- Periodically checking the training progress
- The training will save checkpoints automatically

In [17]:
import time
from datetime import datetime

print('='*70)
print('STARTING TRAINING')
print('='*70)
print(f'Start time: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}')
print(f'Expected duration: ~3-4 hours (100 epochs on T4 GPU)')
print('='*70)
print('\n⏳ Training in progress... Please wait.\n')

start_time = time.time()

try:
    # Start training
    results = model.train(**TRAINING_CONFIG)

    # Calculate duration
    duration = time.time() - start_time
    hours = int(duration // 3600)
    minutes = int((duration % 3600) // 60)

    print('\n' + '='*70)
    print('✓ TRAINING COMPLETED SUCCESSFULLY!')
    print('='*70)
    print(f'End time: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}')
    print(f'Total duration: {hours}h {minutes}m')
    print(f'Results saved to: {RESULTS_DIR}/{TRAINING_CONFIG["name"]}')
    print('='*70)

except KeyboardInterrupt:
    print('\n⚠ Training interrupted by user')
    print('  Partial results may be available in the results directory')

except torch.cuda.OutOfMemoryError:
    print('\n✗ CUDA Out of Memory Error!')
    print('  Solutions:')
    print('  1. Reduce batch size: Change TRAINING_CONFIG["batch"] to 8 or 4')
    print('  2. Reduce image size: Change TRAINING_CONFIG["imgsz"] to 416')
    print('  3. Enable cache=False (already set)')
    print('  4. Restart runtime and try again')
    raise

except Exception as e:
    print(f'\n✗ Training failed: {e}')
    import traceback
    print('\nFull error traceback:')
    traceback.print_exc()
    raise

New https://pypi.org/project/ultralytics/8.3.225 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.200 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=/content/yolov8_cbam/yolov8n-cbam.yaml, data=/content/drive/MyDrive/YOLOv8 Traffic/data.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=2, project=yolov8_cbam_training, name=bd_vehicles_run, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=42, deterministic=False, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1

STARTING TRAINING
Start time: 2025-11-07 06:38:12
Expected duration: ~3-4 hours (100 epochs on T4 GPU)

⏳ Training in progress... Please wait.




                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1       226  CBAM                                         [32]                          
  4                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  5                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  6                  -1  1       610  CBAM                                         [64]                          
  7                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64,

  CBAM initialized: c1=32, reduction_ratio=16, shortcut=True
  CBAM initialized: c1=64, reduction_ratio=16, shortcut=True
  CBAM initialized: c1=128, reduction_ratio=16, shortcut=True
  CBAM initialized: c1=256, reduction_ratio=16, shortcut=True


YOLOv8n-cbam summary: 261 layers, 3024070 parameters, 3024054 gradients, 8.2 GFLOPs

Transferred 367/367 items from pretrained weights
TensorBoard: Start with 'tensorboard --logdir yolov8_cbam_training/bd_vehicles_run', view at http://localhost:6006/
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

...

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: khandaker15-5383 (khandaker15-5383-daffodil-international-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Freezing layer 'model.26.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks skipped ⚠️. Unable to load YOLOv8n due to possible Ultralytics package modifications. Setting 'amp=True'. If you experience zero-mAP or NaN losses you can disable AMP with amp=False.
/usr/local/lib/python3.12/dist-packages/ultralytics/engine/trainer.py:239: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = amp.GradScaler(enabled=self.amp)
train: Scanning /content/drive/.shortcut-targets-by-id/1CA2eiw-QqvOG-A8SZwgNxgr29ai5UAcR/YOLOv8 Traffic/dataset/labels/train.cache... 868 images, 0 backgrounds, 0 corrupt: 100%|██████████| 868/868 [00:00<?, ?it/s]
/usr/local/lib/python3.12/dist-packages/ultralytics/data/augment.py:805: UserWarning: Argument(s) 'quality_lower' are not valid for transform ImageCompression
  A.ImageCompression(quality_lower=75, p=0.0)]  # transforms
/usr/local


✗ Training failed: 'str' object has no attribute '__module__'

Full error traceback:


Traceback (most recent call last):
  File "/tmp/ipython-input-3783686911.py", line 16, in <cell line: 0>
    results = model.train(**TRAINING_CONFIG)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/ultralytics/engine/model.py", line 341, in train
    self.trainer.train()
  File "/usr/local/lib/python3.12/dist-packages/ultralytics/engine/trainer.py", line 192, in train
    self._do_train(world_size)
  File "/usr/local/lib/python3.12/dist-packages/ultralytics/engine/trainer.py", line 413, in _do_train
    self.final_eval()
  File "/usr/local/lib/python3.12/dist-packages/ultralytics/engine/trainer.py", line 558, in final_eval
    strip_optimizer(f)  # strip optimizers
    ^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/ultralytics/utils/torch_utils.py", line 445, in strip_optimizer
    x = torch.load(f, map_location=torch.device('cpu'))
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/

AttributeError: 'str' object has no attribute '__module__'

## 📊 Step 11: Validate Trained Model

Run validation to get final metrics.

In [22]:
print('='*70)
print('MODEL VALIDATION')
print('='*70)

try:
    # Run validation with the data config
    # CRITICAL: Must pass data argument since training already completed
    val_results = model.val(data=DATA_YAML)

    print('\n' + '='*70)
    print('VALIDATION RESULTS')
    print('='*70)
    print(f'mAP50: {val_results.box.map50:.4f} ({val_results.box.map50*100:.2f}%)')
    print(f'mAP50-95: {val_results.box.map:.4f} ({val_results.box.map*100:.2f}%)')
    print(f'Precision: {val_results.box.mp:.4f} ({val_results.box.mp*100:.2f}%)')
    print(f'Recall: {val_results.box.mr:.4f} ({val_results.box.mr*100:.2f}%)')
    print('='*70)

    print('\n✓ Validation completed successfully!')

except Exception as e:
    print(f'✗ Validation failed: {e}')
    import traceback
    traceback.print_exc()


Ultralytics YOLOv8.0.200 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)


MODEL VALIDATION


val: Scanning /content/drive/.shortcut-targets-by-id/1CA2eiw-QqvOG-A8SZwgNxgr29ai5UAcR/YOLOv8 Traffic/dataset/labels/val.cache... 217 images, 0 backgrounds, 0 corrupt: 100%|██████████| 217/217 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:33<00:00,  2.42s/it]
                   all        217        254      0.907      0.938      0.962       0.82
               bicycle        217         17      0.945          1      0.995       0.87
                   bus        217         29      0.896      0.893      0.942      0.834
                   car        217         16          1      0.996      0.995      0.873
                   cng        217         34      0.883      0.971       0.96      0.831
                  auto        217         36       0.91      0.843      0.953       0.84
                  bike        217         25      0.904          1      0.995      0.813
              rickshaw       


VALIDATION RESULTS
mAP50: 0.9623 (96.23%)
mAP50-95: 0.8197 (81.97%)
Precision: 0.9073 (90.73%)
Recall: 0.9380 (93.80%)

✓ Validation completed successfully!


## 💾 Step 12: Copy Results to Google Drive

Save all training results to your Google Drive for safekeeping.

In [25]:
import shutil
from pathlib import Path

print('='*70)
print('COPYING RESULTS TO GOOGLE DRIVE')
print('='*70)

try:
    # Try multiple possible source locations
    possible_sources = [
        f'runs/detect/{TRAINING_CONFIG["project"]}/{TRAINING_CONFIG["name"]}',
        f'{TRAINING_CONFIG["project"]}/{TRAINING_CONFIG["name"]}',
        f'yolov8_cbam_training/bd_vehicles_run',
        'runs/detect/train',
        'runs/detect/train2',
        'runs/detect/train3',
    ]

    # Find the actual results directory
    source_dir = None
    for path in possible_sources:
        if Path(path).exists() and (Path(path) / 'weights').exists():
            source_dir = path
            break

    if source_dir is None:
        # List all directories in runs/detect/ to help debug
        print('⚠ Could not find results directory automatically.')
        print('\nSearching for training results...')

        runs_detect = Path('runs/detect')
        if runs_detect.exists():
            all_runs = sorted(runs_detect.glob('*'), key=lambda p: p.stat().st_mtime, reverse=True)
            print(f'\nFound {len(all_runs)} run(s) in runs/detect/:')
            for i, run_dir in enumerate(all_runs[:5], 1):
                has_weights = (run_dir / 'weights').exists()
                print(f'  {i}. {run_dir.name} {"✓ (has weights)" if has_weights else "✗ (no weights)"}')

            # Use the most recent run with weights
            for run_dir in all_runs:
                if (run_dir / 'weights').exists():
                    source_dir = str(run_dir)
                    print(f'\n✓ Using most recent run: {source_dir}')
                    break

        if source_dir is None:
            raise FileNotFoundError('No training results found. Make sure training completed successfully.')

    # Destination in Google Drive
    run_name = Path(source_dir).name
    dest_dir = f'{DRIVE_PROJECT_PATH}/training_results/{run_name}'

    # Create destination directory
    os.makedirs(dest_dir, exist_ok=True)

    print(f'\nSource: {source_dir}')
    print(f'Destination: {dest_dir}')
    print('\nCopying files...')

    # Copy weights folder
    weights_src = Path(source_dir) / 'weights'
    weights_dst = Path(dest_dir) / 'weights'
    if weights_src.exists():
        shutil.copytree(weights_src, weights_dst, dirs_exist_ok=True)
        # Count weight files
        weight_files = list(weights_dst.glob('*.pt'))
        print(f'✓ Weights copied ({len(weight_files)} files)')
    else:
        print(f'⚠ Weights folder not found at {weights_src}')

    # Copy important files
    files_to_copy = [
        'results.png',
        'results.csv',
        'confusion_matrix.png',
        'confusion_matrix_normalized.png',
        'F1_curve.png',
        'PR_curve.png',
        'P_curve.png',
        'R_curve.png',
        'labels.jpg',
        'labels_correlogram.jpg',
        'args.yaml',
    ]

    copied_count = 0
    for filename in files_to_copy:
        src = Path(source_dir) / filename
        dst = Path(dest_dir) / filename
        if src.exists():
            shutil.copy2(src, dst)
            copied_count += 1

    print(f'✓ Copied {copied_count} result files')

    # Copy validation batch predictions
    val_batches = list(Path(source_dir).glob('val_batch*.jpg'))
    copied_batches = 0
    for val_batch in val_batches[:6]:  # Copy first 6 only
        dst = Path(dest_dir) / val_batch.name
        shutil.copy2(val_batch, dst)
        copied_batches += 1
    print(f'✓ Copied {copied_batches} validation prediction images')

    print('\n' + '='*70)
    print('✓ ALL RESULTS COPIED TO GOOGLE DRIVE!')
    print('='*70)
    print(f'📁 Location: {dest_dir}')
    print(f'\n📊 Key files:')
    print(f'  - Best model: {dest_dir}/weights/best.pt')
    print(f'  - Last checkpoint: {dest_dir}/weights/last.pt')
    print(f'  - Training curves: {dest_dir}/results.png')
    print(f'  - Confusion matrix: {dest_dir}/confusion_matrix.png')
    print('='*70)

except Exception as e:
    print(f'\n⚠ Failed to copy results: {e}')
    print(f'\nTrying to list available directories...')
    try:
        runs_detect = Path('runs/detect')
        if runs_detect.exists():
            all_dirs = list(runs_detect.glob('*'))
            print(f'Directories in runs/detect/: {[d.name for d in all_dirs]}')
    except:
        pass
    import traceback
    traceback.print_exc()


COPYING RESULTS TO GOOGLE DRIVE

Source: yolov8_cbam_training/bd_vehicles_run
Destination: /content/drive/MyDrive/YOLOv8 Traffic/training_results/bd_vehicles_run

Copying files...
✓ Weights copied (2 files)
✓ Copied 4 result files
✓ Copied 0 validation prediction images

✓ ALL RESULTS COPIED TO GOOGLE DRIVE!
📁 Location: /content/drive/MyDrive/YOLOv8 Traffic/training_results/bd_vehicles_run

📊 Key files:
  - Best model: /content/drive/MyDrive/YOLOv8 Traffic/training_results/bd_vehicles_run/weights/best.pt
  - Last checkpoint: /content/drive/MyDrive/YOLOv8 Traffic/training_results/bd_vehicles_run/weights/last.pt
  - Training curves: /content/drive/MyDrive/YOLOv8 Traffic/training_results/bd_vehicles_run/results.png
  - Confusion matrix: /content/drive/MyDrive/YOLOv8 Traffic/training_results/bd_vehicles_run/confusion_matrix.png


## 📥 Step 13: Download Best Model (Optional)

Download the best model weights to your local computer.

In [ ]:
from google.colab import files

best_model_path = f'{dest_dir}/weights/best.pt'
if os.path.exists(best_model_path):
    print(f'Downloading: {best_model_path}')
    files.download(best_model_path)
    print('✓ Download started!')
else:
    print(f'✗ Model not found at: {best_model_path}')

print('Uncomment the code above to download the best model')
print(f'Or access it from Google Drive: {dest_dir}/weights/best.pt')

## 🎯 Step 14: Test Inference (Optional)

Run inference on validation images to see predictions.

In [ ]:
print('='*70)
print('TEST INFERENCE')
print('='*70)

# Load best model
best_model_path = f'{dest_dir}/weights/best.pt'

if os.path.exists(best_model_path):
    print(f'Loading best model from: {best_model_path}')
    best_model = YOLO(best_model_path)
    print('✓ Model loaded')

    # Uncomment to run inference on validation set
    # val_images_path = f'{DATASET_PATH}/{data_config["val"]}'
    #
    # print(f'\nRunning inference on: {val_images_path}')
    # results = best_model.predict(
    #     source=val_images_path,
    #     save=True,
    #     save_txt=True,
    #     conf=0.25,
    #     iou=0.45,
    #     project='/content/predictions',
    #     name='val_predictions',
    #     max_det=300,
    #     augment=False,
    #     agnostic_nms=False,
    # )
    #
    # print('✓ Inference completed')
    # print('📁 Predictions saved to: /content/predictions/val_predictions')

    print('\nUncomment the code above to run inference on validation set')
else:
    print(f'✗ Best model not found at: {best_model_path}')
    print('  Make sure training completed successfully')

## 🎉 Training Complete!

### 📊 What You Have:

1. **Trained Model**: YOLOv8n with CBAM attention mechanism
2. **Results Saved**:
   - Google Drive: `{dest_dir}`
   - Best weights: `weights/best.pt`
   - Training curves: `results.png`
   - Confusion matrix: `confusion_matrix.png`

### 📈 Expected Performance:
- **mAP50**: 98-99% (excellent!)
- **mAP50-95**: 89-91% (very good!)
- **Precision & Recall**: 95-96%

### 🚀 Next Steps:

1. **Review Results**:
   - Check `results.png` for training curves
   - Review `confusion_matrix.png` for per-class performance

2. **Use Your Model**:
   ```python
   from ultralytics import YOLO
   model = YOLO('path/to/best.pt')
   results = model('path/to/image.jpg')
   ```

3. **Export for Deployment**:
   ```python
   model.export(format='onnx')  # For cross-platform
   model.export(format='engine')  # For NVIDIA TensorRT
   ```

### 💡 Tips:
- Results are saved in your Google Drive
- You can disconnect from Colab now
- To resume training, load `weights/last.pt` and set `resume=True`

---

**Thank you for using YOLOv8-CBAM! 🙏**

For questions or issues, check the documentation in the `Attention` folder.

In [23]:
import shutil
from google.colab import files
import os

source_folder = '/content/yolov8_cbam_training'
output_filename = 'yolov8_attention'
zip_file_path = f'/content/{output_filename}.zip'

if os.path.exists(source_folder):
    print(f'Compressing folder: {source_folder}...')
    shutil.make_archive(output_filename, 'zip', source_folder)
    print(f'✓ Folder compressed to: {zip_file_path}')
    print('Initiating download...')
    files.download(zip_file_path)
    print('✓ Download initiated for the zipped folder!')
else:
    print(f'✗ Error: Folder not found at {source_folder}. Please ensure the path is correct.')

Compressing folder: /content/yolov8_cbam_training...
✓ Folder compressed to: /content/yolov8_attention.zip
Initiating download...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✓ Download initiated for the zipped folder!
